# LinkedIn Webscraper

## Part I
Getting Data from LinkedIn

In [6]:
from datetime import datetime
import json
import pandas as pd
import requests
from sqlalchemy import create_engine
from base64 import b64encode
import time

In [7]:
positions = [
    'SAP-Entwickler/SAP-Berater',
    'IT-Projektmanager',
    'Softwareentwickler',
    'Business Intelligence Analyst',
    'IT-Controller',
    'IT-Berater',
    'Produktmanager',
    'App-Entwickler',
    'Anwendungsentwickler',
    'Datenbankspezialist',
    'ERP Manager',
    'Data Analyst',
    'Data Scientist',
    'Data Engineer',
    'Data Visualisation Engineer',
    'Projektmanager',
]

base_url = "https://rest.arbeitsagentur.de/"
jobboerse = "jobboerse/jobsuche-service/pc/v4/jobs?angebotsart=1&was="
page_nav = "&page=1&size=25&pav=false"


In [8]:
base_url = "https://rest.arbeitsagentur.de/"
jobboerse = "jobboerse/jobsuche-service/pc/v4/jobs?angebotsart=1&was="
page_nav = "&page=1&size=25&pav=false"

fertigkeiten_count = 0
joblistings = []
listings_keys = []
for position in positions:
    time.sleep(1)
    joblist_url = f'{base_url}{jobboerse}{position}{page_nav}'
    try:
        response = requests.get(joblist_url)
    except KeyError:
        pass
    json_response = json.loads(response.text)

    i = 0
    
    for listing in json_response['stellenangebote']:
        time.sleep(1)
        joblistings.append(listing)
        refnr = json_response['stellenangebote'][i]['refnr']

        code_bytes = bytes(refnr, 'utf-8')
        encoded_code = b64encode(code_bytes)
        job_id = encoded_code.decode()

        job_url = f"{base_url}jobboerse/jobsuche-service/pc/v2/jobdetails/{job_id}"
        job_response = requests.get(job_url)
        job_json_response = json.loads(job_response.text)
        job_keys = list(job_json_response.keys())
        
        for k in job_keys:
            listings_keys.append(k)
        
        # if 'externeUrl' in job_keys:
        #     pass
        # if 'fertigkeiten' in job_keys:
        #     # print(refnr, '\n')
        #     # print(job_keys, '\n')
        #     # print(job_json_response['fertigkeiten'], '\n')
        #     fertigkeiten_count += 1
        listing.update({'job_id':job_id})
        joblistings.append(listing)
        i += 1

{"stellenangebote": [{"beruf": "Data Engineer", "titel": "Data Analyst (m/w/d)", "refnr": "12288-2893100159-S", "arbeitsort": {"plz": "20355", "ort": "Hamburg", "region": "Hamburg", "land": "Deutschland", "koordinaten": {"lat": 53.5557897, "lon": 9.9825243}}, "arbeitgeber": "Computer Futures, ein Gesch\u00e4ftszweig von SThree", "aktuelleVeroeffentlichungsdatum": "2023-03-03", "modifikationsTimestamp": "2023-03-03T07:40:32.508", "eintrittsdatum": "2023-03-04", "hashId": "UOaKFwCmLpAQ6KuKfEgea1bE2flRNOmVTtevKOIdMLw=", "externeUrl": "https://www.persy.jobs/persy/l/job-aimix-b"}, {"beruf": "Data Scientist", "titel": "Data Analyst*in - befristet f\u00fcr ein Jahr", "refnr": "14164-2023r4kqs2qn-000-S", "arbeitsort": {"plz": "20095", "ort": "Hamburg", "region": "Hamburg", "land": "Deutschland", "koordinaten": {"lat": 53.55067, "lon": 9.9997397}}, "arbeitgeber": "Hamburger Hochbahn AG", "aktuelleVeroeffentlichungsdatum": "2023-03-02", "modifikationsTimestamp": "2023-03-02T13:09:15.652", "eint

In [4]:
df_joblistings = pd.DataFrame(joblistings)
print(df_joblistings)
print(fertigkeiten_count)

## Part II

Saving Data to a Database

In [ ]:
# connection = create_engine('mysql+pymysql://root:@localhost/businessintelligence')
# df_job.to_sql('job', con=connection, index=False, if_exists='append')